# Setup Environment

In [1]:
import os
import sys
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
def init_spark():
    return SparkSession \
        .builder \
        .master("local[*]") \
        .appName("Chess Preprocess") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

spark = init_spark()

In [3]:
seed = 42

# Load Dataset

In [4]:
df = spark.read.csv('data/games.csv', header=True, inferSchema=True)

# Data Inspection

In [5]:
import pandas as pd

In [6]:
df.toPandas().head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


## Inspect Datatypes

In [7]:
df.dtypes

[('id', 'string'),
 ('rated', 'boolean'),
 ('created_at', 'double'),
 ('last_move_at', 'double'),
 ('turns', 'int'),
 ('victory_status', 'string'),
 ('winner', 'string'),
 ('increment_code', 'string'),
 ('white_id', 'string'),
 ('white_rating', 'int'),
 ('black_id', 'string'),
 ('black_rating', 'int'),
 ('moves', 'string'),
 ('opening_eco', 'string'),
 ('opening_name', 'string'),
 ('opening_ply', 'int')]

In [8]:
categorical = [feature for feature, dtype in df.dtypes if dtype in {'string', 'boolean'}]
numerical = [feature for feature, dtype in df.dtypes if dtype in {'double', 'int'}]

### Numerical Features

In [9]:
for col in numerical:
    unique_values = df.select(col).distinct()
    n_unique = unique_values.count()
    if n_unique < 50:
        print(f'{col:20s}:{[row[col] for row in unique_values.collect()]}')
    else:
        print(f'{col:20s}:{n_unique} unique values')

created_at          :13151 unique values
last_move_at        :13186 unique values
turns               :211 unique values
white_rating        :1516 unique values
black_rating        :1521 unique values
opening_ply         :[28, 12, 1, 13, 6, 16, 3, 20, 5, 19, 15, 9, 17, 4, 8, 7, 10, 11, 14, 2, 18, 22, 24]


- `created_at`: Timestamp in UTC
- `last_move_at`: Timestamp in UTC
- `turns`: Number of turns in the match
- `white_rating`: white player rating
- `black_rating`: black player rating
- `opening_ply`: Number of plies used to set up opening

### Categorical Features

In [10]:
for col in categorical:
    unique_values = df.select(col).distinct()
    n_unique = unique_values.count()
    if n_unique < 50:
        print(f'{col:20s}:{[row[col] for row in unique_values.collect()]}')
    else:
        print(f'{col:20s}:{n_unique} unique values')

id                  :19113 unique values
rated               :[True, False]
victory_status      :['resign', 'outoftime', 'mate', 'draw']
winner              :['white', 'black', 'draw']
increment_code      :400 unique values
white_id            :9438 unique values
black_id            :9331 unique values
moves               :18920 unique values
opening_eco         :365 unique values
opening_name        :1477 unique values


- `id`: Game ID, uniquely identifies a match record
- `rated`: If rated, the game result affects player ratings
- `victory_status`: How the game ended
- `winner`: Match winner
- `increment_code`: Game time setting
- `white_id`: white player id
- `black_id`: black player id
- `moves`: Sequence of moves recorded during the match
- `opening_eco`: ECO classification code for the chess openings moves
- `opening_name`: Name of opening moves

## Overview of the Target Columns

In [11]:
df.groupBy('winner').count().show()

+------+-----+
|winner|count|
+------+-----+
| white|10001|
| black| 9107|
|  draw|  950|
+------+-----+



In [12]:
df.withColumn('rating_diff', df.black_rating - df.white_rating).describe('black_rating', 'white_rating', 'rating_diff').show()

+-------+------------------+------------------+------------------+
|summary|      black_rating|      white_rating|       rating_diff|
+-------+------------------+------------------+------------------+
|  count|             20058|             20058|             20058|
|   mean|1588.8319872370128|1596.6318675840064|-7.799880346993718|
| stddev| 291.0361259603342|291.25337573701825|249.03666666007965|
|    min|               789|               784|             -1499|
|    max|              2723|              2700|              1605|
+-------+------------------+------------------+------------------+



## Look for Anomalies

In [13]:
print( f'unique samples / total samples: {df.distinct().count()} / {df.count()} ' )

unique samples / total samples: 19629 / 20058 


In [14]:
print('Count Null Values in each Column')
df.select([F.count(F.when(F.isnull(col), col)).alias(col) for col in df.columns]).toPandas()

Count Null Values in each Column


,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


- There are duplicate rows, which should be dropped from the dataset.
- Luckily, this dataset does not contain missing values.

# Data Preparation (Preprocessing)
Scikit-Learn offers a range of useful methods for preprocessing and data splits. With the approval from the course instructor, we will transform the datasets into *Pandas DataFrames* in this part.

In [15]:
from sklearn.preprocessing import OneHotEncoder
from pyspark.ml.feature import StringIndexer, VectorAssembler

## Fix Anomalies

In [16]:
# Drop Duplicates
df = df.distinct()

## Label `winner`


In [17]:
label_indexer = StringIndexer(inputCol='winner', outputCol='winner_label')
label_indexer_model = label_indexer.fit(df)
df = label_indexer_model.transform(df)
df.select('winner', 'winner_label').distinct().show()
df = df.drop('winner').withColumnRenamed('winner_label', 'winner')

+------+------------+
|winner|winner_label|
+------+------------+
|  draw|         2.0|
| white|         0.0|
| black|         1.0|
+------+------------+



## Feature `increment_code`
As per inspection, the feature contains a string with two numbers separated by `+`. After research, we found that the first number refers to initial total clock time per player in *minutes*; the second number refers to the number of *seconds* added to the total clock time after the player makes a move. We decided to extract these two numbers as two separate features `clock` and `increment` replacing `increment_code`.

In [18]:
df.select('increment_code').head(3)

[Row(increment_code='30+25'),
 Row(increment_code='10+0'),
 Row(increment_code='10+3')]

In [19]:
splits = F.split(df.increment_code, '[+]')
df = df.withColumn('clock', splits.getItem(0).cast('Integer')).withColumn('increment', splits.getItem(1).cast('Integer'))
df.select('clock', 'increment').tail(3)

[Row(clock=5, increment=60),
 Row(clock=8, increment=0),
 Row(clock=3, increment=10)]

## Feature `opening_eco`
As per inspection, each row of this feature is a concatenation of a letter that denotes an [opening moves category](https://www.365chess.com/eco.php). Although there are variations within each category, we assume that the opening moves in each category to be similar enough that we can ignore the differences within each opening move category. Therefore, we extract the first letter from `opening_eco` to a new feature `open_cat` and ignore `opening_eco` during training.

In [20]:
df.select('opening_eco').head(3)

[Row(opening_eco='B11'), Row(opening_eco='B45'), Row(opening_eco='C60')]

In [21]:
df = df.withColumn('open_category', df.opening_eco.substr(0, 1))
df.select('open_category').distinct().show()

+-------------+
|open_category|
+-------------+
|            E|
|            B|
|            D|
|            C|
|            A|
+-------------+



In [22]:
open_cat_indexer = StringIndexer(inputCol='open_category', outputCol='open_cat')
open_cat_model = open_cat_indexer.fit(df)
df = open_cat_model.transform(df)
df.select('open_category', 'open_cat').distinct().show()

+-------------+--------+
|open_category|open_cat|
+-------------+--------+
|            D|     3.0|
|            B|     1.0|
|            E|     4.0|
|            C|     0.0|
|            A|     2.0|
+-------------+--------+



## Feature `rated`
Contains boolean values only and does not require preprocessing. Rated games affect player ratings and may affect performance of the player. We may choose to separate rated games from unrated games for this fact, here we did not.

## Split into Train/Test Sets
1. Encode player ids into numerical values.
    - Ensure that each player is assigned one and only one numerical id.
2. Use encoded player ids to split dataset based on groups.
    - Each group contains the matches played by one player. Make sure that each group are sampled evenly in the training set.

### Scikit-Learn Split methods cannot handle distributed datasets. Because we want to leverage the `GroupSplit` method in Scikit-Learn, we are turning the Spark DF into Pandas DF.

In [23]:
df_pd = df.toPandas()

### Encode categorical player `white_id`s `black_id`s to split based on individual players

In [24]:
player_ids = pd.DataFrame(df_pd[['white_id', 'black_id']])
player_ids = player_ids.stack().pipe(lambda s: pd.Series(pd.factorize(s.values)[0], s.index)).unstack()

df_pd['white_id_num'] = player_ids['white_id']
df_pd['black_id_num'] = player_ids['black_id']

df_pd[['white_id', 'white_id_num', 'black_id', 'black_id_num']]

,white_id,white_id_num,black_id,black_id_num
0,konst767,0,ducksandcats,1
1,everybodylovesjesus,2,ahmd11,3
2,jeff1983,4,bekzodjon,5
3,mellowg7,6,jesteroz,7
4,omnivoid,8,hasan_al-banna,9
...,...,...,...,...
19624,demontechristo66,9751,eie24,2797
19625,r-mohamadi55,15633,eie24,2797
19626,kastorcito,4761,ed84,3240
19627,ssf7,2561,casr,15634


### Data Split

In [25]:
from sklearn.model_selection import GroupShuffleSplit

In [26]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
train_idx, test_idx = next(gss.split(df_pd, df_pd['winner'], groups=df_pd['white_id_num']))

df_train = df_pd.iloc[train_idx]

df_test = df_pd.iloc[test_idx]

In [27]:
# Proof that none of the IDs from the training set are present in the test set.
df_train_values = set(df_train['white_id'].values)
df_test_values = set(df_test['white_id'].values)

assert len(df_train_values.intersection(df_test_values)) == 0, 'A group is present in both training and test sets'

## Feature `victory_status`
From inspection, we observe 4 unique values for this categorical feature: `'resign', 'outoftime', 'mate', 'draw'`. We perform one-hot encoding before feeding this into the model.

In [28]:
enc = OneHotEncoder(sparse=False, dtype=int, handle_unknown='ignore')
enc.fit(df_train['victory_status'].to_numpy().reshape(-1, 1))
encoded_feature_names = 'status_' + enc.categories_[0]
encoded_feature_names

array(['status_draw', 'status_mate', 'status_outoftime', 'status_resign'],
      dtype=object)

In [29]:
one_hot_status = enc.transform(df_train['victory_status'].to_numpy().reshape(-1, 1))
df_train.loc[:,encoded_feature_names] = one_hot_status

one_hot_status = enc.transform(df_test['victory_status'].to_numpy().reshape(-1, 1))
df_test.loc[:,encoded_feature_names] = one_hot_status
df_train[encoded_feature_names].head()

C:\Users\yuxiang\AppData\Local\Temp\ipykernel_18684\2844110880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.loc[:,encoded_feature_names] = one_hot_status
C:\Users\yuxiang\AppData\Local\Temp\ipykernel_18684\2844110880.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.loc[:,encoded_feature_names] = one_hot_status


,status_draw,status_mate,status_outoftime,status_resign
0,0,0,0,1
1,0,0,0,1
3,0,1,0,0
4,0,0,0,1
6,0,0,0,1


## Create Spark DF from Pandas DF Train/Test Set

In [30]:
df_train = spark.createDataFrame(df_train)

df_test = spark.createDataFrame(df_test)

## Feature `moves`
As per inspection, this feature records the list/sequence of moves during the match. Because that the sequence, if parsed correctly, actually indicates the winning player, we need to either drop this feature or obfuscate this information. We find that Spark supports Word2Vec feature transformation, which will turn the sequence into a vector. Such vector representation is independent of the order of the moves, and this can obfuscate part of the information lied within this feature.

In [31]:
from pyspark.ml.feature import Word2Vec

In [32]:
df_train.select('moves').head().moves

'e4 c6 Nc3 d5 Nf3 Bg4 h3 Bh5 exd5 cxd5 Bb5+ Nc6 g4 Bg6 Ne5 Qb6 Nxg6 hxg6 Nxd5 Qc5 Qf3 Rc8 Nc3 e6 d3 Qe5+ Qe4 Bd6 Be3 Ne7 Bd4 Qxe4+ Nxe4 Bb8 Bxg7 Rh4 Bf6 Rh6'

In [33]:
vectorSize = 100
word2Vec = Word2Vec(vectorSize=vectorSize, seed=seed, inputCol="moves_list", outputCol="moves_vec")

Parameters for `Word2Vec`
- `vectorSize`: size of the output vector, the choice of 100 is arbitrary here
- `minCount`: Ignores all words with total frequency lower than this.
- `inputCol`: `moves_list` is the input feature, a list of `moves` split by space
- `outputCol`: `moves_vec` is the output feature, the transformed vector

The Word2Vec model will be trained with the corpus gathered from the `moves` in the training set. After which, it will transform both training and testing datasets' `moves` feature.

In [34]:
# split moves from string into list of strings, moves -> [move, move, ...]
train_moves_corpus = df_train.select(F.split(df_train.moves, '\s', -1).alias('moves_list'))
train_moves_corpus #.collect()[0][0]

DataFrame[moves_list: array<string>]

In [35]:
fitted_word2Vec = word2Vec.fit(train_moves_corpus)
print('Trained Word2Vec Model')
fitted_word2Vec.getVectors().show(5)

Trained Word2Vec Model
+-----+--------------------+
| word|              vector|
+-----+--------------------+
| Bxd2|[0.03784444555640...|
| Nxf6|[-0.0433502122759...|
|  a2+|[-0.0110232504084...|
|Bxd5+|[-0.1207058131694...|
|  Kg8|[-0.2441164255142...|
+-----+--------------------+
only showing top 5 rows



C:\tools\miniconda3\envs\bigdata\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [36]:
'Transformed Moves for the 1st Match in Training Set: ', fitted_word2Vec.transform(train_moves_corpus).head().moves_vec

('Transformed Moves for the 1st Match in Training Set: ',
 DenseVector([0.1189, 0.0334, 0.0764, 0.088, -0.0768, -0.1057, -0.053, 0.0948, -0.0884, 0.1611, -0.0874, 0.0679, -0.0109, 0.0206, -0.0088, 0.0088, 0.015, -0.02, -0.0704, 0.0886, 0.011, -0.0376, 0.0053, -0.0533, -0.0213, -0.0432, -0.0297, -0.1077, -0.0662, -0.0294, -0.0033, 0.0685, -0.0279, -0.0462, 0.1152, 0.0085, -0.0018, 0.1285, -0.0405, -0.0561, -0.0272, -0.0593, 0.0279, -0.073, -0.0607, -0.0779, -0.0512, 0.1071, -0.0009, 0.0791, -0.0241, 0.0591, -0.0326, 0.0043, -0.0869, 0.0118, 0.0989, -0.0697, -0.0492, 0.0123, 0.0241, 0.0614, -0.0494, 0.085, -0.0092, 0.0033, -0.0021, -0.028, -0.0105, -0.0827, -0.0209, 0.0174, 0.0342, 0.0279, -0.1004, 0.081, -0.0158, -0.038, 0.0575, 0.0743, 0.0389, 0.0053, 0.0953, 0.0102, -0.1004, -0.0179, 0.0377, -0.0043, 0.0852, -0.0397, -0.093, -0.0524, 0.1127, 0.0239, -0.0951, -0.1045, 0.0606, -0.0328, 0.064, -0.0075]))

In [37]:
df_train = df_train.withColumn('moves_list', F.split(df_train.moves, '\s', -1))
df_train = fitted_word2Vec.transform(df_train)

df_test = df_test.withColumn('moves_list', F.split(df_test.moves, '\s', -1))
df_test = fitted_word2Vec.transform(df_test)

# Save Datasets

In [38]:
df_train.toPandas().to_csv('data/train.csv', index=False)
df_test.toPandas().to_csv('data/test.csv', index=False)

# Classification Problem
 In a new match between two players, who is most likely to win?

## Feature Selection

In [39]:
features = 'rated', 'turns', 'status_draw', 'status_mate', 'status_resign', 'status_outoftime', 'clock', 'increment', 'white_rating', 'black_rating', 'open_cat', 'opening_ply', 'moves_vec'
target = 'winner'
inputColumns = features + (target,)
inputColumns

('rated',
 'turns',
 'status_draw',
 'status_mate',
 'status_resign',
 'status_outoftime',
 'clock',
 'increment',
 'white_rating',
 'black_rating',
 'open_cat',
 'opening_ply',
 'moves_vec',
 'winner')

In [40]:
training_assembler = VectorAssembler(handleInvalid='error', inputCols=features, outputCol='features')
test_assembler = VectorAssembler(handleInvalid='error', inputCols=features, outputCol='features')

In [41]:
clf_train_df = training_assembler.transform(df_train.select(*inputColumns)).select('features', target)
clf_test_df = test_assembler.transform(df_test.select(*inputColumns)).select('features', target)

## Model Selection

In [42]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

### *Decision Tree* for Classification

In [43]:
dt = DecisionTreeClassifier(labelCol=target, featuresCol="features", seed=seed)

dtParamGrid = (ParamGridBuilder()
               .addGrid(dt.maxDepth, list(range(1, 8)))
               .addGrid(dt.maxBins, [4, 8, 16, 32, 64])
               .addGrid(dt.minInstancesPerNode, list(range(1, 6)))
               .addGrid(dt.impurity, ['gini', 'entropy'])
               .build())

dtEvaluator = MulticlassClassificationEvaluator(labelCol=target, metricName='f1')

From [Spark Docs](https://spark.apache.org/docs/latest/mllib-decision-tree.html#tunable-parameters)
- `maxDepth`: Maximum depth of a tree. Deeper trees are more expressive (potentially allowing higher accuracy), but they are also more costly to train and are more likely to overfit.
- `minInstancesPerNode`: For a node to be split further, each of its children must receive at least this number of training instances. This is commonly used with RandomForest since those are often trained deeper than individual trees.
- `maxBins`: Number of bins used when discretizing continuous features.
- `impurity`: Measure of the homogeneity of the labels at the node. (Choice of `gini` and `entropy` for classification)

#### Hyperparameter Tuning (Grid Search)

In [44]:
dtCv = CrossValidator(estimator = dt,
                      estimatorParamMaps = dtParamGrid,
                      evaluator = dtEvaluator,
                      numFolds = 5)

dtCvModel = dtCv.fit(clf_train_df)

bestParamMap = dtCvModel.bestModel.extractParamMap()

KeyboardInterrupt: 

In [ ]:
# Best Parameters found
print( ','.join([''.join((param.name, '=', repr(value))) for param, value in bestParamMap.items()]) )

#### Model Train

In [ ]:
# Use Best Parameters to initialize and train a separate classifier model
dtBest = DecisionTreeClassifier(cacheNodeIds=False,checkpointInterval=10,featuresCol='features',impurity='gini',labelCol='winner',leafCol='',maxBins=8,maxDepth=6,maxMemoryInMB=256,minInfoGain=0.0,minInstancesPerNode=1,minWeightFractionPerNode=0.0,predictionCol='prediction',probabilityCol='probability',rawPredictionCol='rawPrediction',seed=42)
# assert dtBest.extractParamMap() == bestParamMap, 'Check if the input parameters are the best found. '
bestModel = dtBest.fit(clf_train_df)

#### Model Evaluation

In [ ]:
score = dtEvaluator.evaluate(bestModel.transform(clf_test_df))
'f1 score', score

### *Logistic Regression*

In [ ]:
lr = LogisticRegression(labelCol=target, featuresCol="features")

lrParamGrid = (ParamGridBuilder()
               .addGrid(lr.regParam, [0.01, 0.1, 0.5, 0.8, 2.0])
               .addGrid(lr.elasticNetParam, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
               .addGrid(lr.fitIntercept, [True, False])
               .addGrid(lr.maxIter, [10, 30, 50, 70, 100])
               .addGrid(lr.standardization, [True, False])
               .build())

lrEvaluator = MulticlassClassificationEvaluator(labelCol=target, metricName='f1')

From [Medium Article by Dhiraj Rai](https://dhiraj-p-rai.medium.com/logistic-regression-in-spark-ml-8a95b5f5434c)
- `regParam` the regularization parameter (>= 0). (default: 0.0)
- `elasticNetParam` the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
- `fitIntercept` whether to fit an intercept term. (default: True)
- `maxIter` maximum iteration number.
- `standardization` whether to standardize the training features before fitting the model. The coefficients of models will always be returned on the original scale, so it will be transparent for users. (default: True)

#### Hyperparameter Tuning (Grid Search)

In [ ]:
lrCv = CrossValidator(estimator = lr,
                      estimatorParamMaps = lrParamGrid,
                      evaluator = lrEvaluator,
                      numFolds = 5)

lrCvModel = lrCv.fit(clf_train_df)

bestParamMap = lrCvModel.bestModel.extractParamMap()

In [ ]:
# Best Parameters found
print( ','.join([''.join((param.name, '=', repr(value))) for param, value in bestParamMap.items()]) )

#### Model Train

In [ ]:
lrBest = LogisticRegression(aggregationDepth=2,elasticNetParam=0.0,family='auto',featuresCol='features',fitIntercept=True,labelCol='winner',maxBlockSizeInMB=0.0,maxIter=70,predictionCol='prediction',probabilityCol='probability',rawPredictionCol='rawPrediction',regParam=0.0,standardization=True,threshold=0.5,tol=1e-06)
assert lrBest.extractParamMap() == bestParamMap, 'Check if the input parameters are the best found. '
bestModel = lrBest.fit(clf_train_df)

#### Model Evaluation

In [ ]:
'f1 score', lrEvaluator.evaluate(bestModel.transform(clf_test_df))

In [ ]:
The LogisticRegression performs

# Regression Problem
 Can a player's rating be determined from a single match record with another player?

## Feature Selection

In [ ]:
features = 'rated', 'turns', 'status_draw', 'status_mate', 'status_resign', 'status_outoftime', 'clock', 'increment', 'open_cat', 'opening_ply', 'moves_vec', 'winner', #'white_rating',
target = 'black_rating'
inputColumns = features + (target,)
inputColumns

In [ ]:
training_assembler = VectorAssembler(handleInvalid='error', inputCols=features, outputCol='features')
test_assembler = VectorAssembler(handleInvalid='error', inputCols=features, outputCol='features')

In [ ]:
reg_train_df = training_assembler.transform(df_train.select(*inputColumns)).select('features', target)
reg_test_df = test_assembler.transform(df_test.select(*inputColumns)).select('features', target)

## Model Selection

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

### *Decision Tree* for Regression

In [ ]:
dt = DecisionTreeRegressor(labelCol=target, featuresCol="features", seed=seed)

dtParamGrid = (ParamGridBuilder()
               .addGrid(dt.maxDepth, list(range(1, 8)))
               .addGrid(dt.maxBins, [4, 8, 16, 32, 64])
               .addGrid(dt.minInstancesPerNode, list(range(1, 5 + 1)))
               .build())

dtEvaluator = RegressionEvaluator(labelCol=target, metricName='rmse')

From [Spark Docs](https://spark.apache.org/docs/latest/mllib-decision-tree.html#tunable-parameters)
- `maxDepth`: Maximum depth of a tree. Deeper trees are more expressive (potentially allowing higher accuracy), but they are also more costly to train and are more likely to overfit.
- `minInstancesPerNode`: For a node to be split further, each of its children must receive at least this number of training instances. This is commonly used with RandomForest since those are often trained deeper than individual trees.
- `maxBins`: Number of bins used when discretizing continuous features.
- `impurity`: Impurity measure used to choose between candidate splits. (Only `variance` for regression)

#### Hyperparameter Tuning (Grid Search)

In [ ]:
dtCv = CrossValidator(estimator = dt,
                      estimatorParamMaps = dtParamGrid,
                      evaluator = dtEvaluator,
                      numFolds = 5)

dtCvModel = dtCv.fit(reg_train_df)

bestParamMap = dtCvModel.bestModel.extractParamMap()

In [ ]:
# Best Parameters found
print( ','.join([''.join((param.name, '=', repr(value))) for param, value in bestParamMap.items()]) )

#### Model Train

In [ ]:
# Use Best Parameters to initialize and train a separate classifier model
dtBest = DecisionTreeRegressor(cacheNodeIds=False,checkpointInterval=10,featuresCol='features',impurity='variance',labelCol='black_rating',leafCol='',maxBins=16,maxDepth=5,maxMemoryInMB=256,minInfoGain=0.0,minInstancesPerNode=1,minWeightFractionPerNode=0.0,predictionCol='prediction',seed=42)
# assert dtBest.extractParamMap() == bestParamMap, 'Check if the input parameters are the best found. '
bestModel = dtBest.fit(reg_train_df)

#### Model Evaluation

In [ ]:
score = dtEvaluator.evaluate(bestModel.transform(reg_test_df))
'rmse', score

The RMSE score is very close to the standard deviation of the difference between `black_rating` and `white_rating`. If the model is given the `white_rating` feature, the RMSE would see a decrease of about 40, further improving the model prediction.

### *Linear Regression*

In [ ]:
lr = LinearRegression(labelCol=target, featuresCol="features")

lrParamGrid = (ParamGridBuilder()
               .addGrid(lr.regParam, [0.01, 0.1, 0.5, 0.8, 2.0]).addGrid(lr.solver, ['auto', 'normal', 'l-bfgs'])
               .addGrid(lr.elasticNetParam, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
               .addGrid(lr.fitIntercept, [True, False])
               .addGrid(lr.maxIter, [5, 10, 30, 50, 70, 100])
               .addGrid(lr.standardization, [True, False])
               .build())

lrEvaluator = RegressionEvaluator(labelCol=target, metricName='rmse')

From [Medium Article by Dhiraj Rai](https://dhiraj-p-rai.medium.com/logistic-regression-in-spark-ml-8a95b5f5434c)
- `regParam` the regularization parameter (>= 0). (default: 0.0)
- `elasticNetParam` the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
- `fitIntercept` whether to fit an intercept term. (default: True)
- `maxIter` maximum iteration number.
- `standardization` whether to standardize the training features before fitting the model. The coefficients of models will always be returned on the original scale, so it will be transparent for users. (default: True)

#### Hyperparameter Tuning (Grid Search)

In [ ]:
lrCv = CrossValidator(estimator = lr,
                      estimatorParamMaps = lrParamGrid,
                      evaluator = lrEvaluator,
                      numFolds = 5)

lrCvModel = lrCv.fit(reg_train_df)

bestParamMap = lrCvModel.bestModel.extractParamMap()

In [ ]:
# Best Parameters found
print( ','.join([''.join((param.name, '=', repr(value))) for param, value in bestParamMap.items()]) )

#### Model Train

In [ ]:
lrBest = LinearRegression(aggregationDepth=2,elasticNetParam=0.0,epsilon=1.35,featuresCol='features',fitIntercept=True,labelCol='black_rating',loss='squaredError',maxBlockSizeInMB=0.0,maxIter=5,predictionCol='prediction',regParam=0.1,solver='auto',standardization=False,tol=1e-06)
# assert lrBest.extractParamMap() == bestParamMap, 'Check if the input parameters are the best found. '
bestModel = lrBest.fit(reg_train_df)

#### Model Evaluation

In [ ]:
'rmse score', lrEvaluator.evaluate(bestModel.transform(reg_test_df))

Similarly to DecisionTreeRegressor, the RMSE score is very close to the standard deviation of the difference between `black_rating` and `white_rating`. If the model is given the `white_rating` feature, the RMSE would see a decrease of about 40, further improving the model prediction.